# Spurious Correlations
***
## Introduction
This is a **short** *exploratory data analysis (EDA)* and *visualisation* of two random datasets. Even from this brief analysis, it is apparent that there are underlying issues within the datasets that would need further analysis and cleansing before serious, meaningful insights **could** and/or **should** be inferred. For further discussion on this see [Statistical Disclaimer Section](##statistical-disclaimer)

Ofcourse, these data series are not completely random. I wanted them to have:
 1. Interesting narrative potential
 2. Minimize the need for data cleansing
 
Which led me to picking:
- Tesla Production
    - [Available as estimates from Statista](https://www.statista.com/statistics/715421/tesla-quarterly-vehicle-production/)
    - Total number of units sold
- Nonfarm Business Sector: Real Output Per Hour of All Persons
    - [Available from Federal Reserve Bank of St. Louis](https://fred.stlouisfed.org/series/OPHNFB)
    - Taken as an index (100 being 2012 Q3)
***    
## Data
Both time series datasets are exclusively in the range of 2012 Q3 to 2018 Q1.

### Tesla Production
Given Elon Musk's tendency for secrecy around the inner-workings of Tesla, ([even after a press tour around a section of the manufacturing plant](https://www.cbsnews.com/news/elon-musk-tesla-model-3-problems-interview-today-2018-04-13/)) any specific data for Tesla is hard to come buy. Excluding the publically available financial information.

For this reason my data source is based on estimates calculated from this financial information and fact-checked against Tesla press releases.
    
It represents the *Number of Produced Teslas* of any model since 2012 Q3.

### Real Output per Hour
Based on the measure from "Nonfarm Business Sector: Real Output Per Hour of All Persons", **Nonfarm** is defined by the U.S. Bureau of Labour Statistics as:

>... a subset of the domestic economy and excludes the economic activities of the following: general government, private households, nonprofit organizations serving individuals, and farms. 

In general then and in line with the exceptions stated above, it can be taken to be representative of *productivity per hour* in the U.S. economy.

## Statistical Disclaimer
As stated above, this exploratory analysis is by no means complete and still leaves questions unanswered in these datasets. When this task was given to me, I was told to: 
> - select two random data sets
- undertake a correlation comparison and visualisation [on them]
- add some brief documentation
- spend no more than a few minor hours

In light of the last point in particular, I have stopped myself from carrying out further tests in areas that I believe would require following for these insights to "hold water". This would include tests for:
- The presence of unit roots within the data, which is common in many time series datasets. A good source of information can be found on [Wikipedia](https://en.wikipedia.org/wiki/Unit_root) or [Michael's/Patrick's](https://stats.stackexchange.com/questions/29121/intuitive-explanation-of-unit-root) answers should help for anywone interested in Unit Roots.
- The presence of Heteroskedasticity within the data, shown in the *OLS Residuals* graph at the bottom of this document. This is non-constant variance across a time series, more information at [StatsMakeMeCry.com](http://www.statsmakemecry.com/smmctheblog/confusing-stats-terms-explained-heteroscedasticity-heteroske.html)
- 


## Tools
In the EDA I have utilised *Cufflinks*, *Jupyter Notebooks* (no surprise!), *Pandas*, *Plotly*, and *Statsmodels*.

Jupyter Notebooks was utilised to bring this style of interactive presentation, whilst  still allowing use of Markdown and code snippets. Pandas allows easy and powerful data utilisation and Statsmodels (perhaps not surprisingly) allowed for the running of **OLS** (Ordinary Least Squares) regression and neat presentation of standard output statistics. Cufflinks and Poltly are both visualisation packages which make utilisation of multiple other Python visualisation libraries much easier; such as Matplotlib and Seaborn.

In [1]:
# import matplotlib.pyplot as plt
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf
import statsmodels.api as sm

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

## Imports
Import modules utilised, specifically **Pandas** for data handling and **Plotly** for interactive data visualisation. 

I also have defined options within Plotly that allow for the graphs to be saved and manipulated locally for portability.

In [2]:
df = pd.read_csv("TeslaProduction-BusinessProductivity.csv")

In [3]:
df.shape

(23, 3)

## Data Shape
Showing 23 instances of of data in each dataset, of which there are 3 in total.

In [4]:
df.describe()

,Tesla Production,Real Output per Hour
count,23.000000,23.000000
mean,13925.130435,106.461478
std,9136.266914,1.573489
min,321.000000,104.059000
25%,6674.500000,105.108500
50%,11507.000000,106.668000
75%,22100.000000,107.523500
max,29980.000000,109.050000


## Quick Descriptive Statistics
No need to define these variables further, as they're simplistic enough:
* Few in number, no need to sample
* Scalar in values, not categorical or ordinal

Even though we don't need to sample, best to remember that this is still assumed to be a representative sample of the true distributions of each individual dataset.

In [50]:
df.head()

,Date,Tesla Production,Real Output per Hour
0,2012Q3,321,104.503
1,2012Q4,2400,104.059
2,2013Q1,4900,104.304
3,2013Q2,5150,104.147
4,2013Q3,5500,104.570


In [6]:
df.tail()

,Date,Tesla Production,Real Output per Hour
18,2017Q1,25000,107.683
19,2017Q2,22000,108.142
20,2017Q3,26150,108.849
21,2017Q4,29870,108.943
22,2018Q1,29980,109.050


In [7]:
y0 = df['Tesla Production']
y1 = df['Real Output per Hour']

trace0 = go.Box(
    y=y0,
    name = 'Tesla Production',
    marker = dict(
        color = 'rgb(214, 12, 140)',
        )
    )
trace1 = go.Box(
    y=y1,
    name = 'Real Output per Hour',
    marker = dict(
        color = 'rgb(0, 128, 128)',
        )
    )

data = [trace0, trace1]
iplot(data)

In [10]:
fig0 = df.iplot(
    columns=['Real Output per Hour'], 
    asFigure=True, 
    kind='box', 
    showlegend=False,
    colors = ('#ff0074'),
    )
fig1 = df.iplot(
    columns=['Tesla Production'],
    asFigure=True,
    kind='box', 
    secondary_y=['Tesla Production'],
    showlegend=False,
    title='Comparitive Box Plots',
    yTitle='Real Productivity per Hour ($)',
    secondary_y_title = 'Car Units Produced',
    colors = ('#1b073a'),
)

fig1['data'].extend(fig0['data'])

iplot(fig1)

In [20]:
fig0 = df.iplot(
    columns=['Real Output per Hour', 'Tesla Production'], 
    asFigure=True, 
    kind='histogram', 
    showlegend=False,
    colors = ['#ff0074','#1b073a'],
    title='Distributions',
    xTitle='Real Productivity per Hour ($)',
    yTitle='Frequency',
    histnorm='probability',
    subplots=True,
    shape=(2,1),
    )

iplot(fig0)

In [13]:
fig0 = df.iplot(
    columns=['Real Output per Hour'], 
    asFigure=True, 
    kind='scatter', 
    showlegend=False,
    colors = ('#ff0074'),
    title='Real Output per Hour',
    yTitle='Real Productivity per Hour ($)',
    x='Date'
    )

iplot(fig0)

In [14]:
fig1 = df.iplot(
    columns=['Tesla Production'],
    asFigure=True,
    kind='scatter', 
    showlegend=False,
    title='Teslas Produced',
    yTitle='Car Units Produced',
    colors = ('#1b073a'),
    x='Date'
    )

iplot(fig1)

In [11]:
fig2 = df.iplot(
    columns=['Real Output per Hour'], 
    asFigure=True, 
    kind='scatter', 
    showlegend=False,
    colors = ('#ff0074'),
    x='Date',    
    )
fig3 = df.iplot(
    columns=['Tesla Production'],
    asFigure=True,
    kind='scatter', 
    secondary_y=['Tesla Production'],
    showlegend=False,
    title='Comparitive Time Series',
    yTitle='Real Productivity per Hour ($)',
    secondary_y_title = 'Car Units Produced',
    colors = ('#1b073a'),
    x='Date'
    )

fig3['data'].extend(fig2['data'])

iplot(fig3)

In [51]:
fig2 = df.iplot(
    columns=['Real Output per Hour'], 
    asFigure=True, 
    kind='scatter', 
    showlegend=False,
    colors = ('#ff0074'),
    x='Date',
    mode='markers',
    symbol='diamond'
    )
fig3 = df.iplot(
    asFigure=True,
    kind='scatter', 
    secondary_y=['Tesla Production'],
    showlegend=False,
    title='Comparitive Time Series',
    yTitle='Real Productivity per Hour ($)',
    secondary_y_title = 'Car Units Produced',
    colors = ('#1b073a'),
    x='Date',
    mode='markers',
    )

fig3['data'].extend(fig2['data'])

iplot(fig3)

In [52]:
model = sm.OLS(df['Tesla Production'],df['Real Output per Hour'])

In [53]:
results = model.fit()

In [54]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       Tesla Production   R-squared:                       0.720
Model:                            OLS   Adj. R-squared:                  0.708
Method:                 Least Squares   F-statistic:                     56.64
Date:                Mon, 25 Jun 2018   Prob (F-statistic):           1.60e-07
Time:                        11:44:34   Log-Likelihood:                -241.41
No. Observations:                  23   AIC:                             484.8
Df Residuals:                      22   BIC:                             485.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Real Output per Hour   131.8811 

In [55]:
df_model=df

In [56]:
df['Fitted Values']=results.fittedvalues

In [57]:
df['Residuals']=results.resid

In [58]:
df

,Date,Tesla Production,Real Output per Hour,Fitted Values,Residuals
0,2012Q3,321,104.503,13781.970852,-13460.970852
1,2012Q4,2400,104.059,13723.415643,-11323.415643
2,2013Q1,4900,104.304,13755.726513,-8855.726513
3,2013Q2,5150,104.147,13735.021180,-8585.021180
4,2013Q3,5500,104.570,13790.806886,-8290.806886
5,2013Q4,6892,105.634,13931.128379,-7039.128379
6,2014Q1,6457,104.782,13818.765680,-7361.765680
7,2014Q2,7579,105.435,13904.884040,-6325.884040
8,2014Q3,7785,106.530,14049.293847,-6264.293847
9,2014Q4,9834,105.977,13976.363597,-4142.363597


In [59]:
df['Tesla Production'].sum()

320278

In [60]:
df['Tesla Production'].corr(df['Real Output per Hour'])

0.9140488377327192

In [64]:
fig0 = df.iplot(
    columns=['Fitted Values'], 
    asFigure=True, 
    kind='scatter', 
    showlegend=False,
    colors = ('#ff0074'),
    yTitle='Residuals',
    x='Date',
    title='OLS Fitted Values'
    )

iplot(fig0)

In [63]:
fig0 = df.iplot(
    columns=['Residuals'], 
    asFigure=True, 
    kind='scatter', 
    showlegend=False,
    colors = ('#ff0074'),
    yTitle='Residuals',
    x='Date',
    title='OLS Residuals'
    )

iplot(fig0)

# TODO 
* Cacluate line of best fit (OLS), add to df
* distribution
* R 
* adj. R 
* Pearsons Correlation 
* heteroskedasticity robust 
* Skewness
* Kurtosis
* ones and talk about areas to move on with this stuff